In [ ]:
import requests
import json
import pandas as pd
import numpy as np
import time

In [ ]:
!pip install googlemaps

  Preparing metadata (setup.py) ... done
  Created wheel for googlemaps: filename=googlemaps-4.10.0-py3-none-any.whl size=40715 sha256=c361b6a0b70f24bc3b0fa4c0cae1ac27078eb81b9e38dc7b2252e8bf81341236
  Stored in directory: /root/.cache/pip/wheels/17/f8/79/999d5d37118fd35d7219ef57933eb9d09886c4c4503a800f84
Successfully built googlemaps


In [ ]:
import googlemaps
#Se elimino el api_key para no generar un incidente
gmaps = googlemaps.Client(key=api_key)


In [ ]:
def get_toll_info(api_key, origin, destination):
    url = "https://routes.googleapis.com/directions/v2:computeRoutes"
    headers = {
        'Content-Type': 'application/json',
        'X-Goog-Api-Key': api_key,
        'X-Goog-FieldMask': 'routes.duration,routes.distanceMeters,routes.travelAdvisory.tollInfo,routes.legs.travelAdvisory.tollInfo'
    }
    payload = {
        "origin": {
            "location": {
                "latLng": {
                    "latitude": origin['lat'],
                    "longitude": origin['lng']
                }
            }
        },
        "destination": {
            "location": {
                "latLng": {
                    "latitude": destination['lat'],
                    "longitude": destination['lng']
                }
            }
        },
        "travelMode": "DRIVE",
        "extraComputations": ["TOLLS"],
        "routeModifiers": {
            "vehicleInfo": {
                "emissionType": "GASOLINE"
            },
            "tollPasses": [
                "US_MA_EZPASSMA",  # Ajusta este valor según el tipo de pase de peaje que uses
                "US_WA_GOOD_TO_GO"
            ]
        }
    }

    try:
        response = requests.post(url, headers=headers, data=json.dumps(payload))
        if response.status_code == 200:
            return response.json()
        else:
            print(f"Error en la solicitud: {response.status_code} - {response.text}")
            return None
    except Exception as e:
        print(f"Error al obtener información de peajes entre {origin} y {destination}: {e}")
        return None

# Leer el CSV
ciudades_df = pd.read_csv('/content/capitales_mexico (2).csv')  # Asegúrate de usar el nombre correcto de tu archivo CSV

# Preparar la lista de ciudades
ciudades = []
for index, row in ciudades_df.iterrows():
    ciudad = {
        'nombre': f"{row['Capital']}, {row['Estado']}",
        'lat': row['Latitud'],
        'lng': row['Longitud']
    }
    ciudades.append(ciudad)

# Inicializar matriz para almacenar los resultados
n = len(ciudades)
resultados = np.zeros((n, n))

# Iterar sobre cada par de ciudades
rendimiento = 18.9
valor_hora = 100
gas = 25.19
for i in range(n):
    for j in range(n):
        if i != j:
            origin = {'lat': ciudades[i]['lat'], 'lng': ciudades[i]['lng']}
            destination = {'lat': ciudades[j]['lat'], 'lng': ciudades[j]['lng']}

            # Llamar a la función y obtener datos
            toll_info = get_toll_info(api_key, origin, destination)

            if toll_info and 'routes' in toll_info and toll_info['routes']:
                route = toll_info['routes'][0]

                # Distancia
                distancia_metros = route.get('distanceMeters', 0)
                distancia_km = distancia_metros / 1000

                # Duración
                duracion = route.get('duration', '0s')
                duracion_segundos = float(duracion.replace('s', ''))
                duracion_horas = duracion_segundos / 3600

                # Costo de peajes
                if 'travelAdvisory' in route and 'tollInfo' in route['travelAdvisory']:
                    tolls = route['travelAdvisory']['tollInfo']
                    if 'estimatedPrice' in tolls:
                        costo_peaje = float(tolls['estimatedPrice'][0]['units'])
                    else:
                        costo_peaje = 0
                else:
                    costo_peaje = 0

                # Almacenar los resultados en el diccionario
                resultados[i][j] = ((distancia_km/rendimiento)*gas) + costo_peaje + np.ceil(duracion_horas)*valor_hora
            else:
                # En caso de error o falta de datos, almacenar None
                resultados[i][j] = 0

            # Esperar para cumplir con las políticas de uso de la API
            time.sleep(0.1)
        else:
            # Si es la misma ciudad, asignar 0
            resultados[i][j] = np.inf
from pprint import pprint
np.save('costos.npy', resultados)
print(resultados)


[[           inf  7099.93541534  4040.55702857 ...  3392.84625397
   7318.29974339   400.99101058]
 [ 6826.28899312            inf  3398.1261746  ... 12168.87710159
  10130.21156878  6478.48313016]
 [ 5691.42675873  3398.05020476            inf ...  9729.30823122
  13554.76038783  5243.62222857]
 ...
 [ 3485.43427513  7099.3117381   5462.71597884 ...            inf
   4204.55324233  3076.57957619]
 [ 7356.74002593 12839.95453651  8279.02306243 ...  4248.35505132
             inf  6947.88665979]
 [  405.45723757  6752.39344762  3691.01506085 ...  3893.46561481
   7718.91910423            inf]]
